In [6]:
#  Import libraries and define variables
import os
import shutil
import glob
from tqdm import tqdm
from PIL import Image
import nibabel as nib
import numpy as np
import pandas as pd
import pickle as pkl
import cv2 as cv
import random as rn
from multiprocessing import Pool,Process
import config
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import MaxPooling2D, Dense, Flatten, Dropout, LeakyReLU, Activation, AveragePooling2D, BatchNormalization
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.initializers import HeNormal
from keras.layers.convolutional import Conv2D
import tensorflow as tf
from tensorflow.keras.metrics import TruePositives,TrueNegatives,FalsePositives,FalseNegatives,AUC,Recall,Precision
from tensorflow.keras.callbacks import ModelCheckpoint,ReduceLROnPlateau,EarlyStopping,TensorBoard
from tensorflow.keras.regularizers import l2
import time
import shutil
import sys
sys.path.append('..')


# # Define the modalities and classifications
# modalities = ['T1']
# classifications = ['MGMT_positive', 'MGMT_negative']

# # Define patch size and stride
# block_h, block_w = config.PATCH_SIZE
# stride = 2

# # Interpolated image dimestions
# inter_dim = (110, 90)

# # Define epoch
# epoch = 100
# batch_size = 16

# # Define paths to the BraTS dataset folders
# path = config.MAIN_DIR

# PATH = config.MAIN_DIR + 'Data/'
# Org_Dir = PATH + 'Original_Data_Backup/'
# Work_Dir = PATH + 'Working_Data/'
# Preprocess_Dir = path + 'Preprocessed/layers/'

In [4]:
neg_id =  os.listdir(r'D:\mgmt-patch-model\Data\Working_Data\MGMT_negative\T1')
pos_id = os.listdir(r'D:\mgmt-patch-model\Data\Working_Data\MGMT_positive\T1')

for i in range(len(neg_id)):
    neg_id[i] = 'D:/mgmt-patch-model/Data/Working_Data/MGMT_negative/T1/' +neg_id[i]
for i in range(len(pos_id)):
    pos_id[i] = 'D:/mgmt-patch-model/Data/Working_Data/MGMT_positive/T1/' +pos_id[i]

train_neg_id = neg_id[:60]
val_neg_id = neg_id[60:]

train_pos_id = pos_id[:60]
val_pos_id = pos_id[60:]

In [29]:
def X(lst,l):
    label = []
    train_x = []
    for pat in tqdm(lst):
        for img_id in os.listdir(pat):
            img = cv.imread(os.path.join(pat,img_id),cv.IMREAD_GRAYSCALE)
            if img.shape[0]<30 or img.shape[1]<30: continue

            img = cv.resize(img,(64,64))
            train_x.append(img)
            label.append(l)

    return train_x, label


            

In [30]:
train_pos_x, train_pos_y= X(train_pos_id,1)
train_neg_x, train_neg_y= X(train_neg_id,0)

train_x = train_pos_x + train_neg_x
train_y = train_pos_y + train_neg_y

val_pos_x, val_pos_y= X(val_pos_id,1)
val_neg_x, val_neg_y= X(val_neg_id,0)

val_x = val_pos_x + val_neg_x
val_y = val_pos_y + val_neg_y


100%|██████████| 15/15 [00:00<00:00, 29.08it/s]


In [52]:
train_y.count(0)

2364

In [56]:
def define_model () :
        model = Sequential()

        # model.add(Conv2D(16, (5,5), padding='same',input_shape=(90,110,1),kernel_initializer=HeNormal()))
        # model.add(LeakyReLU(alpha=0.1))
        # model.add(MaxPooling2D(pool_size=(2,2)))
        # model.add(BatchNormalization())
        # model.add(Dropout(0.5))

        model.add(Conv2D(8, (5, 5), padding='same',input_shape=(64,64,1),kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01)))
        model.add(LeakyReLU(alpha=0.1))
        # model.add(MaxPooling2D(pool_size=(2,2)))
        # model.add(BatchNormalization())
        model.add(Dropout(0.1))

        # model.add(Conv2D(8, (3, 3), padding='same',kernel_initializer=HeNormal()))
        # model.add(LeakyReLU(alpha=0.1))
        # model.add(MaxPooling2D(pool_size=(2,2)))
        # model.add(BatchNormalization())
        # model.add(Dropout(0.))

        model.add(Conv2D(4, (3, 3), padding='same',kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01)))
        model.add(LeakyReLU(alpha=0.1))
        # model.add(MaxPooling2D(pool_size=(2,2)))
        # model.add(BatchNormalization())
        # model.add(Dropout(0.1))
        
        model.add(Conv2D(4, (3, 3), padding='same',kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01)))
        model.add(LeakyReLU(alpha=0.1))
        model.add(MaxPooling2D(pool_size=(2,2)))
        model.add(BatchNormalization())
        # model.add(Dropout(0.1))


        # model.add(Conv2D(48, (3, 3), padding='same'))
        # model.add(LeakyReLU(alpha=0.1))
        # model.add(MaxPooling2D(pool_size=(2,2)))
        # model.add(Dropout(0.1))

        model.add(Flatten())  # Convert 3D feature map to 1D feature vector.

        # model.add(Dense(10,kernel_initializer=HeNormal()))
        # model.add(LeakyReLU(alpha=0.1))
        # model.add(BatchNormalization())
        # model.add(Dropout(0.3))

        # model.add(Dense(10,kernel_initializer=HeNormal()))
        # model.add(LeakyReLU(alpha=0.1))
        # model.add(BatchNormalization())
        # model.add(Dropout(0.2))
        
        # model.add(Dense(10,kernel_initializer=HeNormal()))
        # model.add(BatchNormalization())
        # model.add(LeakyReLU(alpha=0.1))
        # model.add(Dropout(0.1))

        model.add(Dense(10,kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01)))
        model.add(BatchNormalization())
        model.add(LeakyReLU(alpha=0.1))
        # model.add(Dropout(0.1))
        model.add(Dense(1, activation='sigmoid'))
        model.compile(loss='binary_crossentropy',
                    optimizer='adam', 
                    metrics=['accuracy',TruePositives(),
                             TrueNegatives(),FalsePositives(),
                             FalseNegatives(),AUC(),Recall(),Precision()])
        return model

In [57]:
model = define_model()

In [58]:
model.fit(np.array(train_x),np.array(train_y), 
          validation_data=(np.array(val_x),np.array(val_y)),
          batch_size= 8,
          epochs = 10,
          shuffle= True
          )

Epoch 1/10
585/585 [==============================] - 20s 31ms/step - loss: 1.0053 - accuracy: 0.5915 - true_positives_7: 1468.0000 - true_negatives_7: 1298.0000 - false_positives_7: 1066.0000 - false_negatives_7: 844.0000 - auc_7: 0.6292 - recall_7: 0.6349 - precision_7: 0.5793 - val_loss: 0.9276 - val_accuracy: 0.4876 - val_true_positives_7: 349.0000 - val_true_negatives_7: 201.0000 - val_false_positives_7: 356.0000 - val_false_negatives_7: 222.0000 - val_auc_7: 0.4557 - val_recall_7: 0.6112 - val_precision_7: 0.4950
Epoch 2/10
585/585 [==============================] - 17s 29ms/step - loss: 0.7689 - accuracy: 0.6730 - true_positives_7: 1523.0000 - true_negatives_7: 1624.0000 - false_positives_7: 740.0000 - false_negatives_7: 789.0000 - auc_7: 0.7399 - recall_7: 0.6587 - precision_7: 0.6730 - val_loss: 0.9482 - val_accuracy: 0.4734 - val_true_positives_7: 90.0000 - val_true_negatives_7: 444.0000 - val_false_positives_7: 113.0000 - val_false_negatives_7: 481.0000 - val_auc_7: 0.4505 -